# Examen

## Datos

**Nombre:** Alan Ruiz Mondragón.  
**Grupo:** 18.

## Librerias

In [ ]:
#Importamos librerias
from  functools import reduce
from  scipy.stats  import  normaltest
from category_encoders.count import CountEncoder
from plotly.offline import plot,iplot
from scipy import stats
from scipy.stats import chisquare
from scipy.stats import ksone
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from varclushi import VarClusHi
import cufflinks as cf
import datetime
import emoji
import jellyfish as jf
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re 
import seaborn as sns
import unicodedata
import warnings
warnings.filterwarnings("ignore")

#Definimos configuraciones
cf.go_offline()
pd.set_option("display.max_columns",200)
pd.set_option("display.max_rows",200)

## Funciones

In [ ]:
#Definimos una función para limpieza de texto
def clean_text(text, pattern="[^a-zA-Z0-9]"):
    text=str(text)
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().lstrip().split())
    return cleaned_text if cleaned_text!="nan" else np.nan

#Definimos una función para revisar la completitud de nuestras variables
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

def unitarias(df,col):
    result=pd.DataFrame(df[col].value_counts(1))
    if result.shape[0]>0:
        if (result[col].values[0]>.91) :
            print(f"{col} -- VARIABLE UNITARIA")

def categoricas(df,col):
    result=pd.DataFrame(df[col].value_counts(1))
    if result.shape[0]>0:
        if (result[col].values[0]>.91) :
            print(f"{col} -- VARIABLE UNITARIA")
        result[col]=result[col].map(lambda x:str(round(x*100,2))+"%")
        result.reset_index(inplace=True)
        result.columns=[col+"_valores","%_aparicion"]
    return result

def bar(df,col,title,x_title="",y_title=""):
    layout = go.Layout(font_family="JetBrains Mono, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"text": x_title,"font": {"family": 'JetBrains Mono, monospace',"size": 18,
        "color": '#000000'}}},yaxis= {"title": {"text": y_title,"font": {"family": 'JetBrains Mono monospace',"size": 18,
        "color": '#000000'}}},title_font_family="JetBrains Mono, monospace",title_font_color="#000000",template="plotly_white")
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='bar',x="conteo",y=col,title=title,asFigure=True,barmode="overlay",sortbars=True,color="#000000",layout=layout)
    fig.update_layout(width=800)
    fig.update_traces(marker_color='#005a96')
    return fig

def pie(df,col,title,x_title="",y_title=""):
    layout = go.Layout(font_family="JetBrains Mono, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"text": x_title,"font": {"family": 'JetBrains Mono, monospace',"size": 18,
        "color": '#000000'}}},yaxis= {"title": {"text": y_title,"font": {"family": 'JetBrains Mono monospace',"size": 18,
        "color": '#000000'}}},title_font_family="JetBrains Mono, monospace",title_font_color="#000000",template="plotly_white")
    #layout = go.Layout(template="plotly_white")
    colors=[ "#581845", "#900c3f","#c70039","#ff5733","#ffc305","#005ba3","#0061a9","#1567af","#226cb6","#2c72bc", "#0061a9","#4c79b7","#7492c6","#98acd4","#bbc7e2","#dde3f1","#ffffff"
]
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='pie',labels="conteo",values=col,title=title,asFigure=True,theme="white")
    
    fig.update_traces(textfont_size=10,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    return fig
    
def box(df,col,title):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='box',title=title,asFigure=True,theme="white",layout=layout,color="#005a96", boxpoints='outliers')
    return fig

def histogram(df,col,bins,title):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='histogram',x=col,bins=bins,title=title,asFigure=True,theme="white",layout=layout,color="#003e6c")
    fig.update_traces(opacity=0.90)
    return fig

def OUTLIERS(df,cols):
    results=pd.DataFrame()
    data_iqr=df.copy()
    data_per=df.copy()
    total=[]
    total_per=[]
    total_z=[]
    indices_=[]

    for col in cols:
        #IQR
        Q1=df[col].quantile(0.25)
        Q3=df[col].quantile(0.75)
        IQR=Q3-Q1
        INF=Q1-1.5*(IQR)
        SUP=Q3+1.5*(IQR)
    
        
        n_outliers=df[(df[col] < INF) | (df[col] > SUP)].shape[0]
        total.append(n_outliers)
        indices_iqr=list(df[(df[col] < INF) | (df[col] > SUP)].index)
        #data_iqr=data_iqr[~(data_iqr[col] < INF) | (data_iqr[col] > SUP)].reset_index(drop=True)
        
        #Percentiles
        INF_pe=np.percentile(df[col].dropna(),5)
    
        SUP_pe=np.percentile(df[col].dropna(),95)
        n_outliers_per=df[(df[col] < INF_pe) | (df[col] > SUP_pe)].shape[0]
        total_per.append(n_outliers_per)
        indices_per=list(df[(df[col] < INF_pe) | (df[col] > SUP_pe)].index)
        #data_per=data_per[~(data_per[col] < INF_pe) | (data_per[col] > SUP_pe)].reset_index(drop=True)
        
        #MEAN CHANGE
        
        #Obtenemos todos los percentiles además del máximo
        perc_100 = [x / 100 for x in range(100)]
        dist = df[col].describe(perc_100).iloc[4:]
        #Obtenemos el cambio entre percentiles
        change_dist = df[col].describe(perc_100).iloc[4:].diff()
        #Obtenemos el cambio promedio entre percentiles
        mean_change = df[col].describe(
            perc_100).iloc[4:].diff().mean()
        #Si el cambio entre el percentil 99 y el maximo es mayor a el cambio promedio entonces:
        if change_dist["max"] > mean_change:
            #La banda superior será el máximo menos el cambio promedio
            ub = dist["max"] - mean_change
            #si la banda superior es más pequeña que el percentil 99 , modificamos la banda para que tome el percentil 99
            if ub < dist["99%"]:
                ub = dist["99%"]
        else:
        #Si el cambio entre el percentil 99 y el maximo es menor o igual a el cambio promedio entonces se toma el percentil 99
            ub = dist["max"]

        if change_dist["1%"] > mean_change:
            lb = dist["0%"] + mean_change
            if lb > dist["1%"]:
                lb = dist["1%"]
        else:
            lb = dist["0%"]
        n_total_z=df[(df[col] < lb) | (df[col] > ub)].shape[0]
        total_z.append(n_total_z)
        indices_z=list(df[(df[col] < lb) | (df[col] > ub)].index)
        
        indices_.append(aux_outliers(indices_iqr,indices_per,indices_z))
        
    results["features"]=cols
    results["n_outliers_IQR"]=total
    results["n_outliers_Percentil"]=total_per
    results["n_outliers_Mean_Change"]=total_z
    results["n_outliers_IQR_%"]=round((results["n_outliers_IQR"]/df.shape[0])*100,2)
    results["n_outliers_Percentil_%"]=round((results["n_outliers_Percentil"]/df.shape[0])*100,2)
    results["n_outliers_Mean_Change_%"]=round((results["n_outliers_Mean_Change"]/df.shape[0])*100,2)
    results["indices"]=indices_
    results["total_outliers"]=results["indices"].map(lambda x:len(x))
    results["%_outliers"]=results["indices"].map(lambda x:round(((len(x)/df.shape[0])*100),2))
    results=results[['features', 'n_outliers_IQR', 'n_outliers_Percentil',
       'n_outliers_Mean_Change', 'n_outliers_IQR_%', 'n_outliers_Percentil_%',
       'n_outliers_Mean_Change_%',  'total_outliers', '%_outliers','indices']]
    return results

def missings_digit(x):
    if sum([y.isdigit() for y in str(x)])>0:
        result=np.nan
    else:
        result=x
    return result

def aux_outliers(a,b,c):
    a=set(a)
    b=set(b)
    c=set(c)
    
    a_=a.intersection(b)

    b_=b.intersection(c)

    c_=a.intersection(c)

    outliers_index=list(set(list(a_)+list(b_)+list(c_)))
    return outliers_index

def chi_square(df,col,valor_miss):
    x_i=df[col].fillna(valor_miss).value_counts()
    k=x_i.sum()
    p_i=df[col].dropna().value_counts(1)
    m_i=k*p_i
    print(x_i)
    print(m_i)
    chi=chisquare(f_obs=x_i,f_exp=m_i)
    p_val=chi.pvalue
    alpha=0.05
    if p_val<alpha:
        print("Rechazamos HO (La porporción de categorias es la misma que la general)")
    else:
        print("Aceptamos HO (La porporción de categorias es la misma que la general)")

## Datasets

In [ ]:
#Cargamos los datos
#Cargamos el dataset principal
df = pd.read_csv('/Users/alanruizmondragon/Documents/PERSONAL/Code/Diplomados/Ciencia de datos/Modulo I/examen/data/data_examen.csv')
#Nombramos al dataset secuendario
df1 = pd.read_csv('/Users/alanruizmondragon/Documents/PERSONAL/Code/Diplomados/Ciencia de datos/Modulo I/examen/data/ocupaciones_examen.csv')

### Créditos

**Contenido**

El primer dataset hace referencia información relacionada con las características de personas que aplicaron para solicitar un crédito, adicional a esa información se cuenta
con una variable ”tgt” que representa una variable objetivo que toma el valor de 1 si se otorgó el crédito y 0 si se rechazó la solicitud.

In [ ]:
#Visualizamos el dataset principal
df.head(5)

**Tamaño**

In [ ]:
#Visualizamos el tamaño del dataset
df.shape

**Formato**

In [ ]:
#Visualizamos la información del dataset
df.info()

**Diccionario**

In [ ]:
#Cargamos el diccionario de datos del dataset principal
diccionario = pd.read_excel('/Users/alanruizmondragon/Documents/PERSONAL/Code/Diplomados/Ciencia de datos/Modulo I/examen/diccionario/VariablesList.xlsx')

In [ ]:
#Visualizamos el diccionario de datos del dataset principal 
diccionario

### Ocupaciones

**Contenido**

El segundo dataset contiene las profesiones y los códigos respectivos a estas.

In [ ]:
#Visualizamos el dataset principal
df1.head(5)

**Tamaño** 

In [ ]:
#Visualizamos el tamaño del dataset
df1.shape

**Formato**

In [ ]:
#Visualizamos la información del dataset
df1.info()

## Calidad de datos

### Definición

La calidad de datos se refiere al grado en que los datos se ajustan a los criterios establecidos para su uso. Estos criterios de calidad de los datos abarcan aspectos como exactitud, coherencia, actualización, exhaustividad y decirigibilidad. La calidad de los datos es uno de los principales factores en el éxito de la base de datos. Si los datos no cumplen con los criterios de calidad adecuados, la información generada a partir de dichos datos también será maliciosa o inútil. Por lo tanto, es crítica la necesidad de vigilancia y mejora de la calidad de los datos.

### Etiquetado de variables

#### Contenido

Visualizamos el contenido y las columnas de los datasets nuevamente para registrar los siguientes pasos de calidad.

##### Dataset princial

In [ ]:
#Visualizamos las columnas contenidas dentro del dataset principal
df.columns

In [ ]:
#Visualizamos el contenido del datset principal
df.info()

In [ ]:
#Visualizamos la distribución del contenido por tipos de datos de la columna del dataset principal 
df.dtypes.value_counts()

**Distribución del contenido**

Los tipos de datos contenidos en el dataset se distribuyen de la siguiente manera

- Columnas de tipo Object: 16
- Columnas de tipo int: 13
- Columnas de tipo float: 3

##### Dataset secundario

In [ ]:
#Visualizamos las columnas contenidas dentro del dataset principal
df1.columns

In [ ]:
#Visualizamos el contenido del datset secundario
df1.info()

In [ ]:
#Visualizamos la distribución del contenido por tipos de datos de la columna del dataset secundario 
df1.dtypes.value_counts()

**Distribución del contenido**

Los tipos de datos contenidos en el dataset se distribuyen de la siguiente manera

- Columnas de tipo int: 1
- Columnas de tipo object: 1

#### Prefijos

Realizamos el etiquetado de las variables utilizando los siguientes prefijos que utilizaremos para los tipos de variables se describen de la siguiente manera

- **c_**: Variables numericas (discretas y continuas).
- **v_**: Variables categoricas.
- **d_**: Variables tipo fecha.
- **t_**: Variables de texto (comentarios, descripciones, url, etc.).
- **g_**: Variables geograficas.

#### Etiquetado

**Dataset principal**

In [ ]:
#Realizamos el etiquetado de las variables del datset principal de acuerdo a los prefijos anteriormente establecidos
c_feats=['ID_CLIENT', 'ID_SHOP', 'AGE', 'QUANT_DEPENDANTS', 'PAYMENT_DAY', 'SHOP_RANK',
        'AREA_CODE_RESIDENCIAL_PHONE', 'MONTHS_IN_RESIDENCE', 'MATE_INCOME', 'PERSONAL_NET_INCOME', 'COD_APPLICATION_BOOTH', 'PROFESSION_CODE',
        'QUANT_ADDITIONAL_CARDS_IN_THE_APPLICATION', 'MONTHS_IN_THE_JOB', 'QUANT_BANKING_ACCOUNTS']
v_feats=['SEX', 'EDUCATION', 'MARITAL_STATUS', 'FLAG_MOTHERS_NAME', 'FLAG_FATHERS_NAME', 'FLAG_RESIDENCE_STATE=WORKING_STATE',
        'FLAG_RESIDENCIAL_ADDRESS=POSTAL_ADDRESS', 'FLAG_OTHER_CARD', 'RESIDENCE_TYPE', 'FLAG_RESIDENCE_TOWN=WORKING_TOWN', 'FLAG_RESIDENCIAL_PHONE', 'FLAG_MOBILE_PHONE', 
        'FLAG_CONTACT_PHONE', 'FLAG_CARD_INSURANCE_OPTION']
t_feats=['PERSONAL_REFERENCE_#1', 'PERSONAL_REFERENCE_#2']

In [ ]:
#Aplicamos 
c_feats_new = ["c_" + x for x in c_feats]
v_feats_new = ["v_" + x for x in v_feats]
t_feats_new = ["t_" + x for x in t_feats]

#Renombramos las columnas
df.rename(columns = dict(zip(c_feats, c_feats_new)), inplace = True)
df.rename(columns = dict(zip(v_feats, v_feats_new)), inplace = True)
df.rename(columns = dict(zip(t_feats, t_feats_new)), inplace = True)

In [ ]:
#Transformamos nuestras columnas en minisculas para una mejor visualización y manipulación futura
df.columns = df.columns.str.lower()

In [ ]:
#Validamos el etiquetado
df.columns

**Dataset secundario**

In [ ]:
df1

In [ ]:
#Realizamos el etiquetado de las variables del datset principal de acuerdo a los prefijos anteriormente establecidos
c_feats = ['PROFESSION_CODE']
v_feats = ['PROFESSION']

In [ ]:
#Aplicamos 
c_feats_new =["c_" + x for x in c_feats]
v_feats_new =["v_" + x for x in v_feats]

#Renombramos las columnas
df1.rename(columns=dict(zip(c_feats, c_feats_new)),inplace=True)
df1.rename(columns=dict(zip(v_feats, v_feats_new)),inplace=True)

In [ ]:
#Transformamos nuestras columnas en minisculas para una mejor visualización y manipulación futura
df1.columns = df1.columns.str.lower()

In [ ]:
#Validamos el etiquetado
df1.columns

### Duplicados

#### General

**Dataset principal**

Con el resultado de siguiente código podemos observar la existencia de **5** valores duplicados en todo el dataset

In [ ]:
#Total de registros duplicados de forma general
df.duplicated().sum()

In [ ]:
#Visualizamos los duplicados 
df[df.duplicated()]

In [ ]:
#Mantenemos el primer elemento de los duplicados
df.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
#Visualizamos el dataset principal y observamos que se han eliminado los duplicados
df

**Dataset secundario**

Con el resultado de siguiente código podemos observar la existencia de **0** valores duplicados en todo el dataset.

In [ ]:
#Total de registros duplicados de forma general
df1.duplicated().sum()

In [ ]:
#Visualizamos el dataset principal y observamos que se han eliminado los duplicados
df1

### Completitud

#### Dataset principal

In [ ]:
df.isnull().sum(0)

In [ ]:
#Revisamos la completitud de las columnas del dataset
completitud(df)

**Observaciones**

Se recomienda que las variables esten completas en al menos 80%, de modo que las que no cumplan con esta condición serán eliminadas. En este sentido, y para el presente análisis, las columnas que no cumplen con dicha regla y que deberían ser eliminadas se enlistan a continuación:

- v_education
- t_personal_reference_#1
- t_personal_reference_#2


In [ ]:
#Revisamos nuevamente el tamaño antes de eliminar las columnas con información incompleta 
df.shape

In [ ]:
#Eliminamos las columnas con información incompleta
df.drop(columns = ["v_education", "t_personal_reference_#1", "t_personal_reference_#2"], inplace = True)

In [ ]:
##Revisamos nuevamente el tamaño despúes de haber de eliminado las columnas con información incompleta 
df.shape

In [ ]:
#Revisamos que no quedan columnas incompletas, es decir, con información menor del 80% 
completitud(df)

#### Dataset secundario

In [ ]:
df1.isnull().sum(0)

In [ ]:
#Revisamos la completitud de las columnas del dataset
completitud(df1)

**Observaciones**

No se encuentran variables incompletas


### Variables fuera de naturaleza

#### Categóricas

In [ ]:
#Revisamos las variables categoricas con la siguiente iteración donde se filtran los valores de las variable de forma única y en un formato de cadena
for i in df.filter(like="v_"):
  print(i)
  values = df[i].astype(str).unique()
  values.sort()
  display(values)
  print("\n")

**Variables a reemplazar con valores NaN**

Al revisar las variables categóricas se pueden observar las siguientes variables con inconsistencias para reemplazra con valores NaN:

1. **v_sex**: Esta variable contiene dos formas diferentes de representación de nulos.
4. **v_residence_type**: Esta variable contiene valores nulos representados como espacios ---> ' '.

**Observaciones**

Las variables **v_marital_status**, **v_flag_residencial_phone** y **v_residence_type**, deberán ser limpiadas más adelante por tener inconsistencias en la presentación de sus categorías.

#### Continuas

In [ ]:
#Revisamos las variables continuas con la siguiente iteración donde se filtran los valores de las variable de forma única y en un formato de cadena
for i in df.filter(like="c_"):
  print(i)
  values = df[i].astype(str).unique()
  values.sort()
  display(values)
  print("\n")

##### Validaciones

Debido a que los valores únicos para las variables continuas son más dificiles de observar haremos validaciones extras de modo que se revisaran casos especiales según la naturaleza de las siguientes variables

**c_age**

**Hallazgos:**

Según **ChatGPT** la persona más longeva de la historia vivió 122 años, de modo que todas las edad registradas mayores a esta edad la catalogaremos como fuera de su naturaleza.

In [ ]:
#Obtenemos descriptivos de la variable
df["c_age"].describe()

In [ ]:
#Obtenemos las observaciones fuera de la naturaleza
df[df["c_age"] > 122]

In [ ]:
#Revisamos el porcentaje que representa la población de más de 122 años que ha solicitado un crédito.
(df[df["c_age"] > 122].shape[0]/df.shape[0]) * 100

**c_months_in_residence**

**Hallazgos:**

1. Dado que la edad máxima para vivir actulamente es de 122 años como se mencionó anteriormente, consideramos el mismo tiempo para los meses vividos en una residencia, es decir, 1464 meses. En este caso, no resultaron haber registros con estas caracteríscas, por lo que no los consideraremos ningun elemento fuera de la naturaleza de la variable, aunque los valores atípicos serán revisados más adelante. 

2. Los meses en la vivienda convertidos en años deberían ser estrictamente menores o iguales a los años de vida, pues no puedes vivir más años en una residencia que de los años que tienes cumplidos. Validamos estos casos realizando una resta entre la variable **c_months_in_residence** y la variable **c_age** y confirmando los casos que resultan negativos. De este modo, aunque a continuación se muestran algunos casos, no consideramos en este momento a ningun valor como fuera de su naturaleza debido que no sabemos cual de las dos variables es la errónea, es posible que en la sección de outliers puedan eliminarse.

In [ ]:
#Obtenemos descriptivos de la variable
df["c_months_in_residence"].describe()

In [ ]:
#Obtenemos las observaciones fuera de la naturaleza y vemos que no hay casos, pero si hay valores atípicos, mismos que serán eliminados en la sección de outliers.
df[df["c_months_in_residence"] > 1464]

In [ ]:
#Obtenemos el porcentaje de datos invalidos debido a la lógica de años con vida y años en vivienda
(df[(df["c_months_in_residence"]).map(lambda x:x>1464)].shape[0]/df.shape[0]) * 100

In [ ]:
#Obtenemos los registros que podrían serinvalidos debido a la lógica de resta entre años con vida y años de vivienda
df[(df["c_age"] - df["c_months_in_residence"]/12).map(lambda x:x<0)]

In [ ]:
#Obtenemos el porcentaje de datos invalidos debido a la lógica de resta entre años con vida y años de vivienda
(df[(df["c_age"] - df["c_months_in_residence"]/12).map(lambda x:x<0)].shape[0]/df.shape[0]) * 100

**c_months_in_the_job**

**Hallazgos:**

1. La Ley Federal del Trabajo en México señala que solo las personas con 15 años o más pueden ofrecer libremente sus servicios profesionales, aunque la situación económica en México resulta dificil para ajustarse a estos marcos, pues la pobreza y desigualdad orilla a miles de mexicanos a trabajar desde que son niños. De este modo, aunque a continuación se muestran algunos casos que podemos identificar con la resta de la variable **c_months_in_the_job** y la variable **c_age**, no consideramos ningun elemento fuera de la naturaleza de la variable, pues no conocemos si hay error en alguna de las dos variables y, pero es posible que en después de revisar la sección de outliers puedan etiquetarse como NaN.

2. Los meses en el trabajo convertidos en años no podrían ser menores a los años de vida, pues no puedes estar en un trabajo más años que de los años que tienes cumplidos. Validamos estos casos realizando una resta entre la variable **c_months_in_the_job** y la variable **c_age** y confirmando los casos que resultan negativos. Debido a que no sabemos cuál es la variable erronea no se harán modificaciones y si estos casos persisten en la sección de outliers los eliminaremos. 

3. Identificamos si hay personas que tengan más de 122 años en su trabajo.

In [ ]:
#Obtenemos descriptivos de la variable
df["c_months_in_the_job"].describe()

In [ ]:
#Observamos los casos de personas que tenian trabajo por debajo de la edad permitida en la ley
df[(df["c_age"] - df["c_months_in_the_job"]/12).map(lambda x:x<15)].head(100)

In [ ]:
#Obtenemos el porcentaje de datos que podrían ser invalidos según la edad con la que se puede trabajar en México
(df[(df["c_age"] - df["c_months_in_the_job"]/12).map(lambda x:x<15)].shape[0]/df.shape[0]) * 100

In [ ]:
#Obtenemos los registros que podrían serinvalidos debido a la lógica de resta entre años con vida y en un trabajo
df[(df["c_age"] - df["c_months_in_the_job"]/12).map(lambda x:x<0)].head(100)

In [ ]:
#Obtenemos el porcentaje de registros que podrían serinvalidos debido a la lógica de resta entre años con vida y en un trabajo
(df[(df["c_age"] - df["c_months_in_the_job"]/12).map(lambda x:x<0)].shape[0]/df.shape[0]) * 100

In [ ]:
#Obtenemos si hay registos donde lleven más de 122 años en el trabajo actual
df[(df["c_months_in_the_job"]/12).map(lambda x:x>1464)].head(100)

**Variables a reemplazar con valores NaN**

Al revisar las variables continuas se pueden observar las siguientes variables con inconsistencias y sus respectivas causas:

1. **c_age**: Esta variable contiene valores con registros de edad fuera del umbral registrado, de modo que los transformamos a NaN.
2. **c_personal_net_income**: Esta variable contiene valores NaN, pero parecen no estar bien definidos, de modo que los variable con el formato incorrecta serán transformadas a NaN.

**Observaciones**

- En la variable **c_months_in_residence** hay registros que no coinciden con la condición de que al ser modificados a años, estos sean menores a los años con vida.
- En la variable **c_months_in_the_job** hay registros que no coinciden con la condición de que al ser modificados a años, estos sean menores a los años con vida y que cumplan con lo establecido por la ley, que es trabajar a partir de los 15 años.

#### Transformación a valores NaN

Convertimos las variales con valores nulos incorrectos a valores NaN.

**Nota**: Las variables que tienen valores fuera de su naturaleza esperamos eliminarlas más adelante en la sección de outliers.

In [ ]:
#v_sex
df['v_sex'] = df['v_sex'].where(df['v_sex'] != 'n.a', np.nan)
display(df['v_sex'].value_counts())

#v_residence_type
df['v_residence_type'] = df['v_residence_type'].where(df['v_residence_type'] != ' ', np.nan)
display(df['v_residence_type'].value_counts())

#c_personal_net_income
df['c_personal_net_income'] = df['c_personal_net_income'].where(df['c_personal_net_income'] != 'n.a.', np.nan)
display(df['c_personal_net_income'].value_counts())

#c_age
df['c_age'] = df['c_age'].where(df['c_age'] < 123, np.nan)
display(df['c_age'].describe())

### Completitud

Revisamos de nuevo la completitud

In [ ]:
#No tenemos variables con una complitud menor al 80%
completitud(df)

### Limpieza y transformación de variables

##### Revisón

**Categóricas**

Basado en lo observado en la salida debajo de esta, procederemos a limpiar las variables:

1. **v_marital_status**
2. **v_flag_residencial_phone**
3. **v_residence_type**

Asimismo, limpiaremos las variables unitarias e identificaremos los tipos de datos correctos del resto de variables categóricas.

In [ ]:
#Revisamos las variables categoricas con la siguiente iteración donde se filtran los valores de las variable de forma única y en un formato de cadena
for i in df.filter(like="v_"):
  print(i)
  values = df[i].astype(str).unique()
  values.sort()
  display(values)
  print("\n")

**Continuas**

Basado en la salida debajo de esta podemos decir que no hay ninguna que tengamos que limpiar, salvo las ya mencionadas con anteriorirdad (c_age, c_months_in_residence, c_months_in_the_job) , así como identificar los tipos.

In [ ]:
#Revisamos las variables continuas con la siguiente iteración donde se filtran los valores de las variable de forma única y en un formato de cadena
for i in df.filter(like="c_"):
  print(i)
  values = df[i].astype(str).unique()
  values.sort()
  display(values)
  print("\n")

#### Limpieza

**v_marital_status**

In [ ]:
#Limpiamos la variable v_marital_status, misma que había sido identificada con anterioridad
df['v_marital_status'].value_counts(1)

In [ ]:
#Aplicamos la limpieza
df["v_marital_status"]=df["v_marital_status"].map(clean_text)
df['v_marital_status'].value_counts(1)

**v_flag_residencial_phone**

In [ ]:
#Limpiamos la variable v_marital_status, misma que había sido identificada con anterioridad
df['v_flag_residencial_phone'].value_counts(1)

In [ ]:
#Aplicamos la limpieza
df["v_flag_residencial_phone"]=df["v_flag_residencial_phone"].map(clean_text)
df['v_flag_residencial_phone'].value_counts(1)

**v_residence_type**

In [ ]:
#Limpiamos la variable v_marital_status, misma que había sido identificada con anterioridad
df['v_residence_type'].value_counts(1)

In [ ]:
#Aplicamos la limpieza
df["v_residence_type"]=df["v_residence_type"].map(clean_text)
df['v_residence_type'].value_counts(1)

**Limpieza homegenizada de todas las variables categóricas**

In [ ]:
for i in df.filter(like="v_"):
    df[i]=df[i].map(clean_text)

**Revisión de la target**

In [ ]:
df["tgt"].value_counts()

#### Unitarias

**Categóricas**

**Eliminación de variables unitarias**

Se muestran las variables que resultaron ser unitarias y su razón de si deberían eliminarse o no.

- **v_flag_mothers_name**: Debe eliminarse ya que al parecer solo indica si el solicitante dio o no el número de su madre, además de que no se encuentra ni el diccionario de datos.

- **v_flag_fathers_name**: Debe eliminarse por la misma razón de la variable anterior.

- **v_flag_residence_state=working_state**: Debe eliminarse, no hay informe de esta variable y es unitaria casi del 100%.

- **v_flag_residencial_address=postal_address**: Debe eliminarse, no hay informe de esta variable y es unitaria casi del 100%.

- **v_flag_other_card**:  Debe eliminarse, es unitaria del 100%.

- **v_flag_mobile_phone**: Debe eliminarse, es unitaria del 100%.

- **v_flag_contact_phone**: Debe eliminarse, es unitaria del 100%.

- **v_flag_card_insurance_option**: Debe eliminarse, es unitaria del 100%.

In [ ]:
#Visualizamos el nombre de las variables unitarias
for col in df.filter(like="v_").columns:
    unitarias(df,col)

In [ ]:
#Visualizamos la distribución de las variables categoricas
for col in df.filter(like="v_").columns:
    display(categoricas(df,col))
    print("\n")

**Continuas**

**Eliminación de variables continuas**

Se muestran las variables que resultaron ser unitarias y su razón de si deberían eliminarse o no.

- **c_quant_dependants**: Debe eliminarse, es unitaria del 100%.

- **c_shop_rank**: Debe eliminarse, no hay informe de esta variable y es unitaria casi del 100%.

- **c_mate_income**: Debe eliminarse, es unitaria casi del 100% y no aporta valor el ingreso de compañero a asignarle un créditp a otra persona.

- **c_quant_banking_accounts**: Debe eliminarse, es unitaria del 100%.

- **c_cod_application_booth**:  Debe eliminarse, es unitaria del 100%.

In [ ]:
#Visualizamos el nombre de las variables unitarias
for col in df.filter(like="c_").columns:
    unitarias(df,col)

In [ ]:
#Visualizamos la distribución de las variables categoricas
for col in df.filter(like="c_").columns:
    display(categoricas(df,col))
    print("\n")

In [ ]:
#Procedemos a eliminar las variables unitarias
df.drop(columns =["v_flag_mothers_name",
                "v_flag_fathers_name",
                "v_flag_residence_state=working_state",
                "v_flag_residencial_address=postal_address",
                "v_flag_other_card","v_flag_mobile_phone",
                "v_flag_contact_phone",
                "v_flag_card_insurance_option",
                "c_quant_dependants",
                "c_shop_rank",
                "c_mate_income",
                "c_quant_banking_accounts",
                "c_cod_application_booth"], inplace=True)

In [ ]:
#Visualizamos nuestra tabla
df.head()

#### Tipos de datos

In [ ]:
#Obtenemos información del dataset, especialmente nos importa el delas columnas
df.info()

**Modificamos continuas** 

In [ ]:
completitud(df)

In [ ]:
df["c_personal_net_income"] = df["c_personal_net_income"].astype(float)

In [ ]:
#Visualizamos nuevamente la información del dataset
df.info()

**Observaciones**

A pesar de que la edad es un valor entero, la dejamos como tipo float para manter la identificación de los valores NaN. 


#### Cruce

In [ ]:
#Dtaframe principal
df.head()

In [ ]:
#Dataframe secundario
df1.head()

In [ ]:
#Cambiamos el tipo de datos de la variable v_profession
df1.info()

In [ ]:
df1["v_profession"] = df1["v_profession"].astype("string")

In [ ]:
#Realizamos un left join
df = df.merge(df1, on = 'c_profession_code', how = 'left')

In [ ]:
#Visualizamos los datos
df.head()

## Análisis Exploratorio de datos

### Categóricas

**Comentarios**

**Variables:**

**v_sex**: La variable muestra que las solicitudes de crédito de este dataset se centran en las **mujeres** con un **69.6%**, mientras que los **hombres** solo el **30.4%**, sin considerar los nulos.

**v_marital_status**: La variable muestra la distribución del estado civil, la mayoría se concentra en las personas **solteras**, es decir, con **50.8%** de la información.

**v_flag_residencial_phone**: La variable muestra la quien de los solicitantes tiene un télefono en casa.

In [ ]:
#Obtenemos descriptivos que nos ayudan al entendimiento del dataset
df.describe(include='O')

In [ ]:
#Obtenemos la visualización de las variables categóricas en gráficas de pastel
for col in df.filter(like="v_"):
    pie(df,col,col).show()

### Continuas

**Comentarios**

Los gráficos de caja nos ayudan a ver como están distribuidas las variables númericas, aqui podemos ver como la veriable **c_age** que representa la edad, tiene outliers que se salen de la caja y de las bandas promedio, tal como mencionamos arriba. De igual forma para las variables **c_months_in_residence**, **c_months_in_the_job** y **c_personal_net_income**, podemos observar los outliers que podrán removerse más adelante.  

In [ ]:
#Obtenemos estadísticos para el entendimeinto de las variable continuas
df.describe().T

In [ ]:
#Obtenemos gráficos de caja para observar la distribución y los outliers de las variables
for col in df.filter(like="c_"):
    box(df,col,col).show()

In [ ]:
for col in df.filter(like="c_"):
    histogram(df,col,12,col).show()

## Datos anómalos

**Histograma antes de la eliminación de Outliers**

Los histogramas que se muestran a continuación parecen demostrar que las variables no se comportan como una normal.

In [ ]:
for col in df.filter(like="c_"):
    histogram(df,col,12,col).show()

**Validación de método a usar**

Para verificar cuál es el método mas adecuado para detectar los outliers necesitamos evaluar la normalidad de nuestras variables continuas y con base en ello, aplicar el mejor método. Esto se debe a que los métodos de Z-Score e IQR, no se adaptan a las series que no sse comportan de manera normal.

Variables no consideradas aplicar para detectar los valores atípicos:

- c_area_code_residencial_phone
- c_payment_day
- c_profession_code
- c_quant_additional_cards_in_the_application



La prueba que usaremos es la más recomendada en clase: **D'Agostino**

In [ ]:
#Aplicamos la prueba de normalidad D'Agostino
for col in df[["c_age", "c_months_in_residence", "c_months_in_the_job", "c_personal_net_income"]].columns:
    print(col)
    stat ,  p  =  normaltest(list(df[col].dropna().values)) 
    alpha = 0.05
    if p > alpha:
        print('La muestra parece una Normal (No se rechaza H0)')
    else:
        print('La muestra no parece una Normal (Se rechaza H0)')

**Método**

Debido a la naturaleza de nuestras variables, las cuales no siguen un distribución normal, utilizaremos el método univariado de percentiles.

In [ ]:
#Aplicamos la función de OUTLIERS la cual muestra un resuen de los diversos métodos, debido a las pruebas realizadas anteriormente, solo mostraremos atención en la prueba de los percentiles
outliers = OUTLIERS(df,list(df[["c_age", "c_months_in_residence", "c_months_in_the_job", "c_personal_net_income"]]))
outliers

In [ ]:
#Guardamos en una lista los indices identificados como atípicos
indices = list(outliers["indices"].values)

In [ ]:
#Guardamos los indicies que nos interesan
indices = list(set(reduce(lambda x,y: x+y, indices)))

In [ ]:
#Obtenemos el porcentaje de elementos que eliminaremos
(len(indices)/df.shape[0])*100

In [ ]:
#Mostramos los valores a imputar
df[df.index.isin(indices)]

In [ ]:
#Obtenemos el tamaño del actual dataframe
df.shape

In [ ]:
df2 = df[~df.index.isin(indices)].reset_index(drop=True)
df2.shape

**Histograma despues de la eliminación de Outliers**

A pesar de haber eliminado los outliers, los histogramas no parecen comportarse como una normal, pero el máximo de edad es de 75 años, casi igual que los requisitos de créditos personales como los de HSBC, BBVA Y Banco Azteca.

In [ ]:
#Obtenemos los histogramas y gráficos de cajas y podemos ver que hay pocos valores atípicos para las variables, incluso el máximo para la edad es de 75
for col in df[["c_age", "c_months_in_residence", "c_months_in_the_job", "c_personal_net_income"]]:
    print(col)
    histogram(df2,col,15,col).show()
    box(df2,col,col).show()
    

## Datos faltantes

In [ ]:
#Revisamos nuevamente los datos faltante
completitud(df2)

### Revisión previa

Debido a las observaciones realizadas en la sección de valores fuera de naturaleza, revisamos de nuevo los valores no validos.

**c_age**

**Nuevas hallazgos:**

La variable ha sido modificada correctamente en outliers y ya no hay valores que colocar como NaN.

In [ ]:
#Obtenemos descriptivos de la variable
df2["c_age"].describe()

In [ ]:
#Obtenemos las observaciones fuera de la naturaleza
df2[df2["c_age"] > 122]

In [ ]:
#Revisamos el porcentaje que representa la población de más de 122 años que ha solicitado un crédito.
(df2[df2["c_age"] > 122].shape[0]/df.shape[0]) * 100

**c_months_in_residence**

**Nuevos hallazgos:**

La variable **c_months_in_residence**, en conjunto con la variable **c_age**, aún muestra inconsistencias, esto aún después de haber pasado por le filtro de outliers, por lo tanto etiquetaremos ambas variables las etiquitaremos como NaN.

In [ ]:
#Obtenemos descriptivos de la variable
df2["c_months_in_residence"].describe()

In [ ]:
#Obtenemos los registros que persisten este error y los cuáles serán modificados como NaN
df2[(df2["c_age"] - df2["c_months_in_residence"]/12).map(lambda x:x<0)].head()

**c_months_in_the_job**

**Nuevos hallazgos:**

Aún hay registros de la variable **c_months_in_the_job** que en conjunto con la variable **c_age** nos muestran que hay personas que trabajaban antes de los 15 años dentro del dataset, debido a la situción social y económica del país, reducire esa cifra a personas que trabajaban antes de los 10 años y se colocarán valores NaN.

Después de haber realizado el ejercicio de outliers pudimos notar que ya no hay inconsistencias entre las variables **c_months_in_the_job** y **c_age**,  pues ya no hay registros de personas con un trabajo con más años de los años que tinen cumplidos en el dataset, de modo que ya no hay nada que convertir en NaN para este caso. 

In [ ]:
#Obtenemos descriptivos de la variable
df2["c_months_in_the_job"].describe()

In [ ]:
#Observamos los casos de personas que tenian trabajo por debajo de la edad fijada como umbral
df2[(df2["c_age"] - df2["c_months_in_the_job"]/12).map(lambda x:x<10)].head()

In [ ]:
#Obtenemos el porcentaje de datos que podrían ser invalidos según la edad con la que se puede trabajar en México
(df2[(df2["c_age"] - df2["c_months_in_the_job"]/12).map(lambda x:x<10)].shape[0]/df2.shape[0]) * 100

In [ ]:
#Obtenemos los registros que podrían ser invalidos debido a la lógica de resta entre años con vida y en un trabajo y vemos que ya no hay registros.
df2[(df2["c_age"] - df2["c_months_in_the_job"]/12).map(lambda x:x<0)].head(100)

### Transformación a valores NaN

In [ ]:
#Modificamos las siguientes variables con valores NaN debido a que no cumplieron los supuestos mencionados con anteriordiad
df2["c_age2"] = df2["c_age"]
df2["c_months_in_residence2"] = df2["c_months_in_residence"]
df2["c_months_in_the_job2"] = df2["c_months_in_the_job"]
df2['c_age']=df2['c_age2'].where(~(df2["c_age2"] - df2["c_months_in_residence2"]/12).map(lambda x:x<0),np.nan)
df2['c_months_in_residence']=df2['c_months_in_residence2'].where(~(df2["c_age2"] - df2["c_months_in_residence2"]/12).map(lambda x:x<0),np.nan)
df2['c_age']=df2['c_age2'].where(~(df2["c_age2"] - df2["c_months_in_the_job2"]/12).map(lambda x:x<10),np.nan)
df2['c_months_in_the_job']=df2['c_months_in_the_job2'].where(~(df2["c_age2"] - df2["c_months_in_the_job2"]/12).map(lambda x:x<10),np.nan)

#Eliminamos las columnas de apoyo creadas
df2 = df2.drop(columns=["c_months_in_the_job2", "c_months_in_residence2", "c_age2"])

In [ ]:
#Validacion de la diferencia negativa entre edad y años en la residencia
df2[(df2["c_age"] - df2["c_months_in_residence"]/12).map(lambda x:x<0)].head()

In [ ]:
df2[(df2["c_age"] - df2["c_months_in_the_job"]/12).map(lambda x:x<10)]

### Revisión e imputación

In [ ]:
#Revisamos nuestras variables con missings
completitud(df2)

**Comentarios**

Imputaremos los valores para cada una de las variables considerando un conjunto de entrenamiento y prueba estratificado, donde el conjunto de prueba tenga el 30% de la información.

In [ ]:
#Guardamos en un dataframe el resumen de completitud y en una lista las variables cuyo valor de missings es menor al 100% para sí identificar las columnas a imputar
missings = completitud(df2)
missings_columns = list(missings[missings["completitud"] < 100]["columna"].values)

In [ ]:
#Mostramis las columnas con missings
missings_columns

**Conjunto de entrenamiento y prueba**

Definimos el conjunto de entrenamiento con el 70% y el de prueba con el 30%.

In [ ]:
#Definimos los conjuntos
X_train, X_test = train_test_split(df2, test_size = 0.30, random_state = 413)

In [ ]:
X_train["v_residence_type"].value_counts(1)

In [ ]:
X_test["v_residence_type"].value_counts(1)

#### Categóricas

Procedemos con la imputación de las variables categóricas basado en la prueba Chi Cuadrada

Para las variables categóricas se utiliza la prueba chi cuadrada para saber si las proporciones de las categorias de la columna antes y después de imputar valores missings se mantienen igual

- k: Número de clases o categorías
- xi: Frecuencias observadas 
- mi: Frecuencias teóricas

- Ho: La proporcion por categoria es igual en la tabla imputada que en la tabla sin missings
- H1: La proporcion por categoria es distinta en la tabla imputada que en la tabla sin missings

In [ ]:
#Asignamos en una lista las columnas categóricas con missings
v_ = list(filter(lambda x: "v_" in x, missings_columns)) 
v_ = [x for x in v_ if x]

In [ ]:
for col in v_:
    print(col)
    display(df2[col].value_counts(1))

In [ ]:
#Revisamos la completitud de nuestras variables en el conjunto de entrenamiento
missings = completitud(X_train)
missings[missings["completitud"]<100]

**v_residence_type**

In [ ]:
X_train["v_residence_type"].value_counts()

In [ ]:
#Obtenemos la moda la variable en cuestión dentro del train
residence_type = X_train["v_residence_type"].mode()[0]
residence_type

In [ ]:
#Obtenemos la frecuencia observada sustiyendo los valores NaN por la moda de la variable a imputar
x_i = X_train["v_residence_type"].fillna(residence_type).value_counts() 
x_i

In [ ]:
#La prueba rechaza HO, de modo que los estadísticos cambian, es decir, si hay un cambio en las proporciones
chi_square(X_train,"v_residence_type", residence_type)

In [ ]:
X_train["v_residence_type"]=X_train["v_residence_type"].fillna(residence_type) 
X_test["v_residence_type"]=X_test["v_residence_type"].fillna(residence_type) 

**v_sex**

In [ ]:
X_train["v_sex"].value_counts()

In [ ]:
#Obtenemos la moda la variable en cuestión dentro del train
sex = X_train["v_sex"].mode()[0]
sex

In [ ]:
#Obtenemos la frecuencia observada sustiyendo los valores NaN por la moda de la variable a imputar
x_i = X_train["v_sex"].fillna(sex).value_counts() 
x_i

In [ ]:
#La prueba acepta HO, de modo que se mantienen las proporciones y continuamos a reemplzar
chi_square(X_train,"v_sex", sex)

In [ ]:
X_train["v_sex"]=X_train["v_sex"].fillna(sex) 
X_test["v_sex"]=X_test["v_sex"].fillna(sex) 

**Completitud**

In [ ]:
#Revisamos la completitud y que se hayan imputado nuestras variables
completitud(X_train) 

#### Continuas

Procedemos con la imputación de las variables categóricas basado en la prueba Kolmogorov

La prueba de Kolmogorov nos sirve para comparar si 2 muestras son de la misma distribución.

Al imputar valores missings de variables numéricas se busca no afectar la distribución original de los datos, por ello se debe de buscar el valor correcto que se sustituirá en lugar de los missings y que no nos modifique la distribución original.

- H0 (hipótesis nula): dos muestras son de la misma distribución
- H1 (hipótesis alternativa): dos muestras provienen de distribuciones diferentes
- Si el estadístico KS es pequeño o el valor p es alto, entonces no podemos rechazar HO

In [ ]:
#Actualizamos la variable de missings y filtramos las que faltan por imputar
missings = completitud(X_train)
missings[missings["completitud"]<100]

**c_age**

El resultado que se presenta a continuación concluye en que la mejor variable para imputador los datos en **c_age** es la mediana, debdio a que el estadístico de prueba fue el menor y con un p-value mayor que el resto.

In [ ]:
#Obtenemos la moda, media y mediana del conjunto train
mean = X_train["c_age"].mean() 
median = X_train["c_age"].median() 
mode = X_train["c_age"].mode()[0] 

In [ ]:
mean

In [ ]:
median

In [ ]:
mode

In [ ]:
originales=list(X_train["c_age"].dropna().values)
imputados=list(X_train["c_age"].fillna(mean).values)
stats.ks_2samp(originales,imputados)

In [ ]:
originales=list(X_train["c_age"].dropna().values)
imputados=list(X_train["c_age"].fillna(median).values) 
stats.ks_2samp(originales,imputados)

In [ ]:
#Se obtiene el valor crítico que se compara con la mediana
#Podemos decir que ambas vienen la misma distribución
ksone.ppf(1-.05/2, len(originales))

In [ ]:
originales=list(X_train["c_age"].dropna().values)
imputados=list(X_train["c_age"].fillna(mode).values) 
stats.ks_2samp(originales,imputados)

In [ ]:
#Como resultado de las pruebas imputamos por la mediana
imp = SimpleImputer(missing_values = np.nan, strategy="median") 

In [ ]:
#Imputamos
imp.fit(X_train[["c_age"]])

In [ ]:
#Terminamos de imputar
X_train[["c_age"]]=imp.transform(X_train[["c_age"]])
X_test[["c_age"]]=imp.transform(X_test[["c_age"]])

In [ ]:
missings = completitud(X_train)
missings[missings["completitud"]<100]

**c_months_in_residence**

El resultado que se presenta a continuación concluye en que la mejor variable para imputador los datos en **c_months_in_residence** es la mediana, debdio a que el estadístico de prueba fue el menor y con un p-value mayor que el resto.

In [ ]:
#Obtenemos la moda, media y mediana del conjunto train
mean = X_train["c_months_in_residence"].mean() 
median = X_train["c_months_in_residence"].median() 
mode = X_train["c_months_in_residence"].mode()[0] 

In [ ]:
mean

In [ ]:
median

In [ ]:
mode

In [ ]:
originales=list(X_train["c_months_in_residence"].dropna().values)
imputados=list(X_train["c_months_in_residence"].fillna(mean).values)
stats.ks_2samp(originales,imputados)

In [ ]:
originales=list(X_train["c_months_in_residence"].dropna().values)
imputados=list(X_train["c_months_in_residence"].fillna(median).values) 
stats.ks_2samp(originales,imputados)

In [ ]:
originales=list(X_train["c_months_in_residence"].dropna().values)
imputados=list(X_train["c_months_in_residence"].fillna(mode).values) 
stats.ks_2samp(originales,imputados)

In [ ]:
#Se obtiene el valor crítico que se compara con la mediana
#Podemos decir que ambas vienen la misma distribución
ksone.ppf(1-.05/2, len(originales))

In [ ]:
#Como resultado de las pruebas imputamos por la mediana
imp = SimpleImputer(missing_values = np.nan, strategy="median") 

In [ ]:
#Imputamos
imp.fit(X_train[["c_months_in_residence"]])

In [ ]:
#Terminamos de imputar
X_train[["c_months_in_residence"]] = imp.transform(X_train[["c_months_in_residence"]])
X_test[["c_months_in_residence"]] = imp.transform(X_test[["c_months_in_residence"]])

In [ ]:
missings = completitud(X_train)
missings[missings["completitud"]<100]

**c_personal_net_income**

El resultado que se presenta a continuación concluye en que la mejor variable para imputador los datos en **c_personal_net_income** es la mediana, debdio a que el estadístico de prueba fue el menor y con un p-value mayor que el resto.

In [ ]:
#Obtenemos la moda, media y mediana del conjunto train
mean = X_train["c_personal_net_income"].mean() 
median = X_train["c_personal_net_income"].median() 
mode = X_train["c_personal_net_income"].mode()[0] 

In [ ]:
mean

In [ ]:
median

In [ ]:
mode

In [ ]:
originales=list(X_train["c_personal_net_income"].dropna().values)
imputados=list(X_train["c_personal_net_income"].fillna(mean).values)
stats.ks_2samp(originales,imputados)

In [ ]:
originales=list(X_train["c_personal_net_income"].dropna().values)
imputados=list(X_train["c_personal_net_income"].fillna(median).values) 
stats.ks_2samp(originales,imputados)

In [ ]:
originales=list(X_train["c_personal_net_income"].dropna().values)
imputados=list(X_train["c_personal_net_income"].fillna(mode).values) 
stats.ks_2samp(originales,imputados)

In [ ]:
#Se obtiene el valor crítico que se compara con la mediana
#Podemos decir que ambas vienen la misma distribución
ksone.ppf(1-.05/2, len(originales))

In [ ]:
#Como resultado de las pruebas imputamos por la mediana
imp = SimpleImputer(missing_values = np.nan, strategy="median") 

In [ ]:
#Imputamos
imp.fit(X_train[["c_personal_net_income"]])

In [ ]:
#Terminamos de imputar
X_train[["c_personal_net_income"]] = imp.transform(X_train[["c_personal_net_income"]])
X_test[["c_personal_net_income"]] = imp.transform(X_test[["c_personal_net_income"]])

In [ ]:
missings = completitud(X_train)
missings[missings["completitud"]<100]

**c_months_in_the_job**

El resultado que se presenta a continuación concluye en que la mejor variable para imputador los datos en **c_months_in_the_job** es la mediana y la moda, debdio a que el estadístico de prueba fue el menor y con un p-value mayor que el resto.

In [ ]:
#Obtenemos la moda, media y mediana del conjunto train
mean = X_train["c_months_in_the_job"].mean() 
median = X_train["c_months_in_the_job"].median() 
mode = X_train["c_months_in_the_job"].mode()[0] 

In [ ]:
mean

In [ ]:
median

In [ ]:
mode

In [ ]:
originales=list(X_train["c_months_in_the_job"].dropna().values)
imputados=list(X_train["c_months_in_the_job"].fillna(mean).values)
stats.ks_2samp(originales,imputados)

In [ ]:
originales=list(X_train["c_months_in_the_job"].dropna().values)
imputados=list(X_train["c_months_in_the_job"].fillna(median).values) 
stats.ks_2samp(originales,imputados)

In [ ]:
originales=list(X_train["c_months_in_the_job"].dropna().values)
imputados=list(X_train["c_months_in_the_job"].fillna(mode).values) 
stats.ks_2samp(originales,imputados)

In [ ]:
#Se obtiene el valor crítico que se compara con la mediana
#Podemos decir que ambas vienen la misma distribución
ksone.ppf(1-.05/2, len(originales))

In [ ]:
#Como resultado de las pruebas imputamos por la mediana
imp = SimpleImputer(missing_values = np.nan, strategy="median") 

In [ ]:
#Imputamos
imp.fit(X_train[["c_months_in_the_job"]])

In [ ]:
#Terminamos de imputar
X_train[["c_months_in_the_job"]] = imp.transform(X_train[["c_months_in_the_job"]])
X_test[["c_months_in_the_job"]] = imp.transform(X_test[["c_months_in_the_job"]])

In [ ]:
missings = completitud(X_train)
missings[missings["completitud"]<100]

**Completitud**

In [ ]:
completitud(X_train)

In [ ]:
completitud(X_test)

**Arreglos**

Modificamos los tipos de datos ahora que tenemos una completitud del 100% en todos los datos. Esto se hará para el train y el test.

In [ ]:
X_train.info()

In [ ]:
#Enteros
X_train["c_age"] = X_train["c_age"].astype(int)
X_train["c_months_in_residence"] = X_train["c_months_in_residence"].astype(int)
X_train["c_months_in_the_job"] = X_train["c_months_in_the_job"].astype(int)
X_test["c_age"] = X_test["c_age"].astype(int)
X_test["c_months_in_residence"] = X_test["c_months_in_residence"].astype(int)
X_test["c_months_in_the_job"] = X_test["c_months_in_the_job"].astype(int)

#Cadenas
X_train["v_sex"] = X_train["v_sex"].astype("string")
X_train["v_marital_status"] = X_train["v_marital_status"].astype("string")
X_train["v_flag_residencial_phone"] = X_train["v_flag_residencial_phone"].astype("string")
X_train["v_residence_type"] = X_train["v_residence_type"].astype("string")
X_train["v_flag_residence_town=working_town"] = X_train["v_flag_residence_town=working_town"].astype("string")
X_train["v_profession"] = X_train["v_profession"].astype("string")
X_test["v_sex"] = X_test["v_sex"].astype("string")
X_test["v_marital_status"] = X_test["v_marital_status"].astype("string")
X_test["v_flag_residencial_phone"] = X_test["v_flag_residencial_phone"].astype("string")
X_test["v_residence_type"] = X_test["v_residence_type"].astype("string")
X_test["v_flag_residence_town=working_town"] = X_test["v_flag_residence_town=working_town"].astype("string")
X_test["v_profession"] = X_test["v_profession"].astype("string")

In [ ]:
X_train.info()

In [ ]:
X_test.info()

## Ingenieria de variables

**Revisión**

Revisamos el dataset de prueba y de test.

In [ ]:
X_train.head()

In [ ]:
X_test.head()

### Categóricas

In [ ]:
#Revisamos nuestras variables categóricas
for i in X_test.filter(like="v_"):
  print(i)

#### One-Hot Encoding

Decidí crear nuevas variables con One-Hot Encodig tomando como base las variables categoricas dicotomicas, es decir, para la variable **v_sex** y las variables **v_flag_residencial_phone**, **v_flag_residence_town**. Esto genera 1 o 0 depesndiendo de la categoría.

In [ ]:
#Eriquetamos a la variable de sexo como 1 si es femenino o 0 si es masculino
X_train["v_sex_female"] = X_train["v_sex"].map(lambda x: 1 if x == "f" else 0)
X_test["v_sex_female"] = X_test["v_sex"].map(lambda x: 1 if x == "f" else 0)

#Eriquetamos a la variable que tiene como bandera si se tiene teléfono en la residencia, asigandole 1 si tiene y 0 si no tiene
X_train["v_flag_residencial_phone_y"] = X_train["v_sex"].map(lambda x: 1 if x == "y" else 0)
X_test["v_flag_residencial_phone_y"] = X_test["v_sex"].map(lambda x: 1 if x == "y" else 0)

#Eriquetamos a la variable que tiene como bandera si trabaja en la ciudad, asigandole 1 si es así y 0 si no tiene
X_train["v_flag_residence_town=working_town_y"] = X_train["v_flag_residence_town=working_town"].map(lambda x: 1 if x == "y" else 0)
X_test["v_flag_residence_town=working_town_y"] = X_test["v_flag_residence_town=working_town"].map(lambda x: 1 if x == "y" else 0)

#### Count Encoding

Con Count Encoding asignamos la frecuencia absoluta de las categórias a las que se le asigna esta ingeniería. En este caso revisamos para la situación civil y el tipo de residencia. 

In [ ]:
X_train["v_marital_status"].value_counts()

In [ ]:
X_test["v_marital_status"].value_counts()

In [ ]:
X_train["v_residence_type"].value_counts()

In [ ]:
X_test["v_residence_type"].value_counts()

In [ ]:
marital_count = X_train['v_marital_status'].value_counts()
residence_count = X_train['v_residence_type'].value_counts()

In [ ]:
marital_count.to_dict()

In [ ]:
residence_count.to_dict()

In [ ]:
X_train['v_marital_status_count_enc'] = X_train['v_marital_status'].map(marital_count.to_dict())
X_train['v_residence_type_count_enc'] = X_train['v_residence_type'].map(residence_count.to_dict())

X_test['v_marital_status_count_enc'] = X_test['v_marital_status'].map(marital_count.to_dict())
X_test['v_residence_type_count_enc'] = X_test['v_residence_type'].map(residence_count.to_dict())

In [ ]:
X_train.head()

In [ ]:
X_test.head()

#### Target encoding

Con target encoding agruparemos por una variable y asiganremos el promedio de la misma basado en un variable categórica, en este caso, usaremos a la variable categórica **v_profession** agrupada por la media del ingreso neto. La lógica de estoes guardar el promedio del ingreso de las profesiones de las personas que solicitan un crédito.

In [ ]:
X_train["v_profession"].value_counts()

In [ ]:
v_profession_means = X_train.groupby('v_profession')['c_personal_net_income'].mean()

In [ ]:
X_train['v_profession_means_target_enc'] = X_train['v_profession'].map(v_profession_means)

X_test['v_profession_means_target_enc'] = X_test['v_profession'].map(v_profession_means)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

### Nuevas características

Realizamos limpieza de la variable categórica **v_profession** creando una nueva variable, ademas agregamos diversas creaciónes de características de medidas de texto.

**Limpieza**

In [ ]:
#Limpiamos la varibale y quitamos stop words
X_train["v_profession_clean"] = X_train["v_profession"].map(clean_text)
X_test["v_profession_clean"] = X_test["v_profession"].map(clean_text)

In [ ]:
#Desacrgamos las stop words en inglés para quedarnos con las palabras importantes de la categoría
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words("english")

In [ ]:
X_train["v_profession_clean"]=X_train["v_profession_clean"].map(lambda text:" ".join([x for x in text.split(" ") if x not in stop_words]))
X_test["v_profession_clean"]=X_test["v_profession_clean"].map(lambda text:" ".join([x for x in text.split(" ") if x not in stop_words]))

In [ ]:
X_train.head()

In [ ]:
X_test.head()

**Tokenización**

In [ ]:
#Convertimpos en lista al texto de la variable categórica de profesión para crear nuevas caraterísticas y mejorar la información del modelo
X_train["v_profession_clean"]=X_train["v_profession_clean"].map(lambda x:x.split())
X_test["v_profession_clean"]=X_test["v_profession_clean"].map(lambda x:x.split())

**Longitud**

In [ ]:
#Longitud del train
#Palabras
X_train['c_numero_palabras'] = X_train["v_profession_clean"].apply(lambda x: len(x))
#Letras
X_train['c_numero_letras'] = X_train["v_profession_clean"].map(lambda x: sum(len(word) for word in x))
#Longitud
X_train['c_long_pro_pala'] = X_train['c_numero_letras'] / X_train['c_numero_palabras']

#Longitud del test
#Palabras
X_test['c_numero_palabras'] = X_test["v_profession_clean"].apply(lambda x: len(x))
#Letras
X_test['c_numero_letras'] = X_test["v_profession_clean"].map(lambda x: sum(len(word) for word in x))
#Longitud
X_test['c_long_pro_pala'] = X_test['c_numero_letras'] / X_test['c_numero_palabras']

In [ ]:
X_train.head()

In [ ]:
X_test.head()

**Count Vectorized**

In [ ]:
#Realizamos el procedimiento de count vectorized selecionando 
vect = CountVectorizer(stop_words="english",ngram_range=(1,5))
vect.fit(X_train["v_profession"])
array_train = vect.transform(X_train["v_profession"])
array_test = vect.transform(X_test["v_profession"])

In [ ]:
aux_train=pd.DataFrame(array_train.toarray(),columns=vect.get_feature_names_out())
aux_test=pd.DataFrame(array_test.toarray(),columns=vect.get_feature_names_out())

aux_train = aux_train.rename(columns={col: "c_" + col for col in aux_train.columns})
aux_test = aux_test.rename(columns={col: "c_" + col for col in aux_test.columns})

In [ ]:
aux_train

In [ ]:
aux_test

In [ ]:
#Añadimoms las caracteristicas creadas al train
X_train=pd.concat([X_train.reset_index(drop=True),aux_train.reset_index(drop=True)],axis=1)

#Añadimoms las caracteristicas creadas al test
X_test=pd.concat([X_test.reset_index(drop=True),aux_test.reset_index(drop=True)],axis=1)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_train.info()

### Continuas

Crearemos nuevas características polinomiales con las variables continuas, que se enlistan a continuación:

- **c_age**
- **c_months_in_residence**
- **c_months_in_the_job**
- **c_personal_net_income**

Seleccionamos solo estas continuas debido a su naturaleza, no encontre sentido en agregar variables polinomiales a los id's como el de cliente o a los fierentes códigos del dataset.


In [ ]:
variables = (df[['c_age', 'c_months_in_residence', 'c_months_in_the_job', 'c_personal_net_income']].columns)

In [ ]:
variables

In [ ]:
poly = PolynomialFeatures(2)

In [ ]:
cont_car_train = X_train.copy()
cont_car_test = X_test.copy()

In [ ]:
cont_car_train = poly.fit_transform(cont_car_train[variables])
cont_car_test = poly.fit_transform(cont_car_test[variables])

In [ ]:
cont_car_train = pd.DataFrame(columns=poly.get_feature_names_out(), data = cont_car_train)
cont_car_test = pd.DataFrame(columns=poly.get_feature_names_out(), data = cont_car_test)

In [ ]:
cont_car_train = cont_car_train.iloc[:,5:]
cont_car_train.head()

In [ ]:
cont_car_test = cont_car_test.iloc[:,5:]
cont_car_test.head()

In [ ]:
#Añadimoms las caracteristicas creadas al train
X_train = pd.concat([X_train.reset_index(drop=True), cont_car_train.reset_index(drop=True)],axis=1)

#Añadimoms las caracteristicas creadas al test
X_test = pd.concat([X_test.reset_index(drop=True), cont_car_test.reset_index(drop=True)],axis=1)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_train.info()

## Reducción de dimensiones

##### VARClus

Con el algoritmo de VARClus aplicamos PCA, y realiza diversas selecciones a través del cálculo de los compnentes N PCA, para posteriormente asignar de manera iterativa la Clasificación de componentes (NCS). Después de NCS, el algoritmo intenta aumentar la varianza total explicada por la primera Componente PCA de cada clúster secundario mediante la reasignación de funciones entre clústeres.

**X_train**

In [ ]:
#Seleccionamos las características creadas
variables_pca = X_train.iloc[:,27:]

In [ ]:
#Definimos a vc
vc = VarClusHi(variables_pca,maxeigval2=.7)
vc.varclus()

In [ ]:
#Mostramos los estadísticos
vc.rsquare

In [ ]:
#Mostramos las variables seleccionadas y sus estadísticos
vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")

In [ ]:
#Obtenemos las variables consideradas
variables_consideradas =list(vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")["Variable"].values)
variables_consideradas

In [ ]:
#Obtenemos las variables eliminadas
variables_no_consideradas = list(variables_pca.drop(variables_consideradas, axis=1))
variables_no_consideradas

In [ ]:
print("variables_consideradas")
display(variables_consideradas)
display("Total de variables_consideradas", len(variables_consideradas))

print("variables_no_consideradas")
display(variables_no_consideradas)
display("Total de variables_consideradas", len(variables_no_consideradas))

### Tabla final

In [ ]:
#Pegamos las variables caracteísticas selccionadas
X_train = pd.concat([X_train.reset_index(drop=True), variables_pca.drop(variables_no_consideradas, axis=1).reset_index(drop=True)],axis=1)

In [ ]:
#Tabla train final
X_train.head()

## Conclusiones finales


Aunque a lo largo del ejercicio se mencionaron las diversas observaciones que habían en cada una de las secciones, es importante concluir de manera general. El dataset presentaba pocas inconsistencias en cuanto a la calidad de datos, se realizaron diversos arreglos en cuanto a los registros de variables continuas y categóricas, especialmente pudimos observar errores en las cadenas de las variables categóricas que se arreglaron limpiando el texto. 

Asimismo, pudimos observar en las variables continuas diversas inconsistencias entre las edades y las variables que representaban meses en la vivienda y meses en el trabajo actual, algunas de estas inconsistencias se arreglaron en la sección de outliers y los fenómenos que persistían fueron reemplazados por valores **NaN** debido al bajo nivel de porcentaje que este representaba. Siguiente a esto se hicieron evaluaciones con pruebas para eliminar los valores atípicos y pudimos notar que la distintas variables continuas no seguían una distribución normal, por lo que descartamos realizar Z-score e IQR y acudimos por el método de Percentiles. Posteriormente utilizamos Chi-cuadrada y la prueba Kolmogorov para la imputación de valores ausentes, las variables a las que se redujó el dataset por características unitarias permitió un manejo más sencillo de la aplicación de los métodos mencionados.


Finalmente, se crearon características mediante la ingeniería de variables y diversos arreglos que permiten mejorar el desarrollo futuro de modelos de Machine Learning, dichas características fueron evaluadas por el algoritmo VARClus que contiene una evaluación de PCA, y seleccionamos las variables finales a utilizar para nuestro conjunto de entrenamiento.